In [1]:
folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/permutations'

In [2]:
# %%bash

! mkdir -p $folder/barplots
!mkdir $folder/tidy
!mkdir $folder/size
!mkdir $folder/psi
# mkdir $folder/modalities_sizes

mkdir: cannot create directory `/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/permutations/tidy': File exists
mkdir: cannot create directory `/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/permutations/size': File exists
mkdir: cannot create directory `/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/permutations/psi': File exists


In [3]:
base_folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/permutations'


In [4]:
! mkdir -p $base_folder

In [5]:
%%file ~/randomly_permute_and_calculate_modality.py

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import flotilla
import os
import sys
import anchor

sns.set(style='ticks', context='paper', rc={'font.sans-serif':'Arial', 'pdf.fonttype': 42})

# --- Parse command line arguments --- #
# Ensure the iteration is always at least 1
iteration = max(int(sys.argv[1]), 1)
iteration_str = str(iteration).zfill(4)
suffix = 'iteration_{}'.format(iteration_str)
# seed = (sum(ord(c) for c in 'randomly_permute_modalities')/iteration) % 5437
np.random.seed(iteration)



# --- Set up study --- #
base_folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/permutations'

flotilla_dir = '/projects/ps-yeolab/obotvinnik/flotilla_projects/'

study = flotilla.embark('singlecell_pnm_figure1_supplementary_post_splicing_filtering', 
                        flotilla_dir=flotilla_dir)

not_outliers = study.splicing.singles.index.difference(study.splicing.outliers.index)
splicing_singles_no_outliers = study.splicing.singles.loc[not_outliers]

print splicing_singles_no_outliers.shape
splicing_singles_no_outliers = splicing_singles_no_outliers.groupby(
    study.sample_id_to_phenotype).apply(lambda x: x.dropna(thresh=10, axis=1))
print splicing_singles_no_outliers.shape


# To permute the whole array, have to do it on the flattened values and then reshape them 
# because otherwise numpy only shuffles the rows which is not what we want
permuted_psi = splicing_singles_no_outliers.groupby(study.sample_id_to_phenotype).apply(
    lambda x: pd.DataFrame(np.random.permutation(x.values.flat).reshape(x.shape), index=x.index, columns=x.columns))
permuted_psi.to_csv('{}/psi/{}.csv'.format(base_folder, suffix))

bayesian = anchor.BayesianModalities()
modality_assignments = permuted_psi.groupby(study.sample_id_to_phenotype).apply(bayesian.fit_predict)

modality_assignments[splicing_singles_no_outliers.groupby(study.sample_id_to_phenotype).count() < 20] = np.nan
modalities_tidy = modality_assignments.stack().reset_index()
modalities_tidy = modalities_tidy.rename(columns={'level_1': 'event_name', 0: "modality",})
modalities_tidy.to_csv('{}/tidy/{}.csv'.format(base_folder, suffix), index=False)

sizes = modalities_tidy.groupby(['phenotype', 'modality']).size()
sizes.to_csv('{}/size/{}.csv'.format(base_folder, suffix))


g = anchor.barplot(modalities_tidy, x='phenotype', hue='modality', size=2)
g.savefig('{}/barplots/{}.pdf'.format(base_folder, suffix))

Overwriting /home/obotvinnik/randomly_permute_and_calculate_modality.py


In [6]:
from qtools import Submitter

base_command = 'python ~/randomly_permute_and_calculate_modality.py'

commands = []
for i in range(1000):
    commands.append('{} {}'.format(base_command, i))
Submitter(commands, 'randomly_permute_modality', walltime='1:00:00', array=True, queue='home-scrm')

running 500 tasks as an array-job.
Wrote commands to randomly_permute_modality1.sh
.Submitted script to queue home-scrm. Job ID: 4996454
running 500 tasks as an array-job.
Wrote commands to randomly_permute_modality2.sh
.Submitted script to queue home-scrm. Job ID: 4996455


In [10]:
! grep -i walltime randomly_permute_modality*out*